### STEP-1: Import Libraries

In [3]:
import numpy as np 
import pandas as pd 

import time

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn import preprocessing

import joblib
from sklearn.preprocessing import MaxAbsScaler

import warnings
warnings.filterwarnings("ignore")

### STEP-2: EDA

#### **2.1 Upload Data**

In [4]:
# from google.colab import files #upload friday working hours files - count 3
# files.upload()

#### **2.2 Load Data**

In [5]:
df1 = pd.read_csv("CICIDS2017/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df2=pd.read_csv("CICIDS2017/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df3=pd.read_csv("CICIDS2017/Friday-WorkingHours-Morning.pcap_ISCX.csv")

#append data
nRowsRead = None
df = pd.concat([df1,df2])
del df1,df2
df = pd.concat([df,df3])
del df3
nRow, nCol = df.shape
print(f'Data: {nRow} rows {nCol} columns')

Data: 703245 rows 79 columns


#### **2.3 View data**

In [6]:
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


#### **2.4 View type of attacks**

In [7]:
df[' Label'].value_counts()

BENIGN      414322
PortScan    158930
DDoS        128027
Bot           1966
Name:  Label, dtype: int64

### STEP-3: Preprocessing

#### **3.1 Remove special characters & drop features with low correlation***

In [8]:
integer = []
f = []
for i in df.columns[:-1]:
    if df[i].dtype == "int64": integer.append(i)
    else : f.append(i)

df[integer] = df[integer].astype("int32")
df[f] = df[f].astype("float32")

one_variable_list = []
for i in df.columns:
    if df[i].value_counts().nunique() < 2:
        one_variable_list.append(i)
df.drop(one_variable_list,axis=1,inplace=True)
df.columns =  df.columns.str.strip()
# drop nan and infinite rows
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

# merging similar classes with low instances
df["Label"] = df["Label"].replace(["Web Attack � Brute Force","Web Attack � XSS","Web Attack � Sql Injection"],"Web Attack")
# drop duplicate rows
df =  df.drop_duplicates(keep="first")
df.reset_index(drop=True,inplace=True)

#feature reduction 
#dropping very high correlated features 
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
df =  df.drop(to_drop, axis=1)
df.shape

(615433, 43)

#### **3.2 View processed data**

In [9]:
df

,Destination Port,Flow Duration,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Min,Flow Bytes/s,...,Init_Win_bytes_forward,Init_Win_bytes_backward,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Std,Idle Min,Label
0,54865,3,2,12,6,6,6.0,0,0,4.000000e+06,...,33,-1,20,0.0,0.0,0,0,0.0,0,BENIGN
1,55054,109,1,6,6,6,6.0,6,6,1.100917e+05,...,29,256,20,0.0,0.0,0,0,0.0,0,BENIGN
2,55055,52,1,6,6,6,6.0,6,6,2.307692e+05,...,29,256,20,0.0,0.0,0,0,0.0,0,BENIGN
3,46236,34,1,6,6,6,6.0,6,6,3.529412e+05,...,31,329,20,0.0,0.0,0,0,0.0,0,BENIGN
4,54863,3,2,12,6,6,6.0,0,0,4.000000e+06,...,32,-1,20,0.0,0.0,0,0,0.0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615428,53,61452,4,180,45,45,45.0,177,177,8.689709e+03,...,-1,-1,20,0.0,0.0,0,0,0.0,0,BENIGN
615429,53,171,2,80,40,40,40.0,136,136,2.058480e+06,...,-1,-1,32,0.0,0.0,0,0,0.0,0,BENIGN
615430,53,222,2,90,45,45,45.0,177,177,2.000000e+06,...,-1,-1,32,0.0,0.0,0,0,0.0,0,BENIGN
615431,123,16842,1,48,48,48,48.0,48,48,5.700036e+03,...,-1,-1,20,0.0,0.0,0,0,0.0,0,BENIGN


### STEP-4: Data Preparation

#### **4.1 Save processed data**

In [10]:
df = df[:615000]
df.to_csv(r'clean_data.csv', index = True, header=True)

real_data = df[-433:]

#del real_data['Label']

real_data.to_csv(r'real_time.csv', index = True, header=True)

#### **4.2 Encode data for modeling**

In [11]:
catFeatureslist = ['Label']

import csv

dataset = df

for cf1 in catFeatureslist:
    le =preprocessing.LabelEncoder()
    le.fit(dataset[cf1].unique())
    actual = dataset[cf1].unique()
    dataset[cf1] = le.transform(dataset[cf1])

dataset.to_csv (r'encoded.csv', index = True, header=True)

#### **4.3 Split data for validation, real-time predictions & Save Encoded data**

In [12]:
df = pd.read_csv('clean_data.csv')
df = df.rename(columns={"Unnamed: 0": "AttackId"}) 
df.to_csv('clean_data.csv')

df1 = pd.read_csv('real_time.csv')
df1 = df1.rename(columns={"Unnamed: 0": "AttackId"}) 
df1.to_csv('real_time.csv')

df2 = pd.read_csv('encoded.csv')
df2 = df2.rename(columns={"Unnamed: 0": "AttackId"}) 
df2.to_csv('encoded.csv')

### STEP-5: Training Phase

#### **5.1 Read clean data**

In [13]:
df = pd.read_csv('clean_data.csv')
del df['Unnamed: 0']

#### **5.2 Data Size**

In [14]:
nRow, nCol = df.shape
print(f'Data: {nRow} rows {nCol} columns')

Data: 615000 rows 44 columns


#### **5.3 View clean data**

In [15]:
df.head()

,AttackId,Destination Port,Flow Duration,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Min,...,Init_Win_bytes_forward,Init_Win_bytes_backward,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Std,Idle Min,Label
0,0,54865,3,2,12,6,6,6.0,0,0,...,33,-1,20,0.0,0.0,0,0,0.0,0,BENIGN
1,1,55054,109,1,6,6,6,6.0,6,6,...,29,256,20,0.0,0.0,0,0,0.0,0,BENIGN
2,2,55055,52,1,6,6,6,6.0,6,6,...,29,256,20,0.0,0.0,0,0,0.0,0,BENIGN
3,3,46236,34,1,6,6,6,6.0,6,6,...,31,329,20,0.0,0.0,0,0,0.0,0,BENIGN
4,4,54863,3,2,12,6,6,6.0,0,0,...,32,-1,20,0.0,0.0,0,0,0.0,0,BENIGN


In [16]:
x = df.drop(["Label"],axis=1)
y = df["Label"]

print(y.unique())


data_len = len(df)

test_size = int(data_len/2) #50% of the total data 

train_range = int(data_len - test_size) #70% of the total data

train = df[:train_range]

test = df[train_range:data_len]

train.to_csv('train.csv',index=False)
test.to_csv('test.csv',index=False)

X_train = x[:train_range]
X_test = x[train_range:data_len]

Y_train = y[:train_range]
Y_test = y[train_range:data_len]


['BENIGN' 'DDoS' 'PortScan' 'Bot']


In [17]:
scaler = MaxAbsScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#### **5.4 Classify Attack using KNN** 

##### **5.4.1 Metric: Euclidean**

In [18]:
startKNN = time.time()
best_KNN_Classifier = KNeighborsClassifier(n_jobs=-1, n_neighbors=6, metric='euclidean', algorithm='auto') 
best_KNN_Classifier.fit(X_train, Y_train);

print("Training time(KNN+euclidean):"+str((time.time()-startKNN)/60)+" mins")

Y_pred = best_KNN_Classifier.predict(X_test)

confidence = metrics.accuracy_score(Y_test, Y_pred)
print("Confidence (%):", confidence*100) 
print()

Training time(KNN+euclidean):0.5621208628018697 mins
Confidence (%): 85.38341463414633



##### **5.4.2 Metric: minkowski**

In [19]:
startKNN = time.time()
KNN_Classifier = KNeighborsClassifier(n_jobs=-1, n_neighbors=6, metric='minkowski', algorithm='auto') 
KNN_Classifier.fit(X_train, Y_train);

print("Training time(KNN+minkowski):"+str((time.time()-startKNN)/60)+" mins")

Y_pred = KNN_Classifier.predict(X_test)

confidence = metrics.accuracy_score(Y_test, Y_pred)
print("Confidence (%):", confidence*100) 
print()

Training time(KNN+minkowski):0.6425613005956013 mins
Confidence (%): 85.38341463414633



##### **5.4.3 Metric: chebyshev**

In [20]:
startKNN = time.time()
KNN_Classifier = KNeighborsClassifier(n_jobs=-1, n_neighbors=6, metric='chebyshev', algorithm='auto') 
KNN_Classifier.fit(X_train, Y_train);

print("Training time(KNN+chebyshev):"+str((time.time()-startKNN)/60)+" mins")

Y_pred = KNN_Classifier.predict(X_test)

confidence = metrics.accuracy_score(Y_test, Y_pred)
print("Confidence (%):", confidence*100) 
print()

Training time(KNN+chebyshev):0.5685390154520671 mins
Confidence (%): 74.80650406504064



##### **5.4.4 Metric: manhattan**

In [21]:
startKNN = time.time()
KNN_Classifier = KNeighborsClassifier(n_jobs=-1, n_neighbors=6, metric='manhattan', algorithm='auto') 
KNN_Classifier.fit(X_train, Y_train);

print("Training time(KNN+manhattan):"+str((time.time()-startKNN)/60)+" mins")

Y_pred = KNN_Classifier.predict(X_test)

confidence = metrics.accuracy_score(Y_test, Y_pred)
print("Confidence (%):", confidence*100) 
print()

Training time(KNN+manhattan):0.5552995165189107 mins
Confidence (%): 85.37853658536585



#### **5.5 Save best performing model for real-time classification**

In [22]:
#KNN(euclidean + minkowski) performed equally well.
filename = 'knnModel_Euclidean.sav'
joblib.dump(best_KNN_Classifier, filename)
  
loaded_model = joblib.load(filename)

### STEP-6: Application Phase (SafeML + Reinforcement learning)

##### **6.1 Load training & real-time validation data**

In [23]:
real_time = pd.read_csv("real_time.csv")
all_data = pd.read_csv("clean_data.csv")

x = real_time.drop(["Label"],axis=1).fillna(0)
y = real_time["Label"]
del x['Unnamed: 0']

x

,AttackId,Destination Port,Flow Duration,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Min,...,Down/Up Ratio,Init_Win_bytes_forward,Init_Win_bytes_backward,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Std,Idle Min
0,614567,53,215,2,76,38,38,38.0,242,242,...,1,-1,-1,20,0.0,0.0,0,0,0.0,0
1,614568,53,837,2,74,37,37,37.0,77,77,...,1,-1,-1,20,0.0,0.0,0,0,0.0,0
2,614569,443,14336951,1,0,0,0,0.0,0,0,...,5,8192,42780,32,6336908.0,0.0,6336908,6336908,0.0,8000043
3,614570,53,31040,2,82,41,41,41.0,113,113,...,1,-1,-1,32,0.0,0.0,0,0,0.0,0
4,614571,53,148,2,76,38,38,38.0,54,54,...,1,-1,-1,20,0.0,0.0,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,614995,53,30477,1,79,79,79,79.0,145,145,...,1,-1,-1,32,0.0,0.0,0,0,0.0,0
429,614996,53,30572,1,46,46,46,46.0,74,74,...,1,-1,-1,32,0.0,0.0,0,0,0.0,0
430,614997,53,30473,1,60,60,60,60.0,76,76,...,1,-1,-1,20,0.0,0.0,0,0,0.0,0
431,614998,53,31267,1,48,48,48,48.0,113,113,...,1,-1,-1,20,0.0,0.0,0,0,0.0,0


##### **6.2 Load best model for real-time attack classification**

In [24]:
import joblib

filename = 'knnModel_Euclidean.sav'
  
loaded_model = joblib.load(filename)


In [25]:
Y_pred = loaded_model.predict(x)

In [26]:
Y_pred

array(['BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN',
       'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN',
       'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'PortScan',
       'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN',
       'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN',
       'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN',
       'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN',
       'PortScan', 'BENIGN', 'BENIGN', 'PortScan', 'BENIGN', 'BENIGN',
       'BENIGN', 'PortScan', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN',
       'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN',
       'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN',
       'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN',
       'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN',
       'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN',
       'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN', 'BENIGN

In [27]:
y

0      BENIGN
1      BENIGN
2      BENIGN
3      BENIGN
4      BENIGN
        ...  
428    BENIGN
429    BENIGN
430    BENIGN
431    BENIGN
432    BENIGN
Name: Label, Length: 433, dtype: object

###### **6.2.1 Make predictions**

In [28]:
confidence = metrics.accuracy_score(y, Y_pred)
print("Confidence (%):", confidence*100) 
print()

Confidence (%): 97.22863741339492



###### **6.2.2 Check confidence against threshhold & Alert human if 'low confidence'**

In [29]:
#Set Confidence Threshold = 100%
Confidence_Threshold = 100

if(confidence < Confidence_Threshold):
    print('Alert human')

Alert human


###### **6.2.3 Print misclassified attacks**

In [30]:
#Print items not correctly classified
import pandas as pd 
# intialise data of lists. 
data = {'AttackId':x['AttackId'], 'Actual':y, 'Predicted':Y_pred} 
  
# Create DataFrame 
df = pd.DataFrame(data) 

In [31]:
not_classified_attacks = df[df['Actual']!=df['Predicted']]
not_classified_attacks

,AttackId,Actual,Predicted
17,614584,BENIGN,PortScan
42,614609,BENIGN,PortScan
45,614612,BENIGN,PortScan
49,614616,BENIGN,PortScan
198,614765,Bot,BENIGN
199,614766,Bot,BENIGN
203,614770,Bot,BENIGN
204,614771,Bot,BENIGN
206,614773,Bot,BENIGN
226,614793,Bot,BENIGN


###### **6.2.4 Assist human to find similar attacks (Fidning nearest neighbouring using 'Euclidean' distance metric)**

In [32]:
import math 
#function calculates distance start
def euclidean_distnce(data, p1, p2):
    common_item = {}
    
    for item in data[p1]:
        if item in data[p2]:
            common_item[item] = True

    if len(common_item) == 0: return 0

    #calculate Euclidean distance
    #√((x1-x2)^2 + (y1-y2)^2)
    
    distance = sum([math.pow(data[p1][itm] - data[p2][itm], 2) for itm in common_item.keys()])
    distance = math.sqrt(distance)
    #return result
    
    return 1/(distance + 1)

def calculateSimilarItems(data, attackId, n):   
    result = {}
    data_reverse = data 
    
    item = attackId
    
    #finding distance score of all other items with respect to current item
    
    similarities = [(euclidean_distnce(data_reverse, item, other), other) for other in data_reverse.keys() if item == attackId] 
    similarities.sort()   
    result[item] = similarities[0:n]   
    return result

In [33]:
attack = not_classified_attacks.iloc[0]

#read encoded insurance data.
df = pd.read_csv(r'encoded.csv')

del df['Unnamed: 0']

###### **6.2.5 Print similar attacks for human agent to review**

In [34]:

df.set_index("AttackId", drop=True, inplace=True)

#format data to find similar claims.
dictionary = df.to_dict(orient="index")

In [35]:
#dictionary
##attackId to query.
attackId = attack['AttackId']

#No.of.similar attacks to query.
cnt_similar_claims = 10

similar_attacks = calculateSimilarItems(dictionary,attackId,cnt_similar_claims)

attacks = similar_attacks.get(attackId)

similar_attack_ids=[] 
for similar_attack in range(len(attacks)):
    similar_attack_ids.append(attacks[similar_attack][1])

df = all_data[all_data['AttackId'].isin(similar_attack_ids)]

attacks = all_data[all_data['AttackId']==attackId]
attacks
#to replace with real-time data which has to be separated from all_training data.

,Unnamed: 0,AttackId,Destination Port,Flow Duration,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,...,Init_Win_bytes_forward,Init_Win_bytes_backward,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Std,Idle Min,Label
614584,614584,614584,53176,11492,4,24,6,6,6.0,0,...,180,-1,20,0.0,0.0,0,0,0.0,0,BENIGN


###### **6.2.6 Human to classify 'low confidence' attack manually**

In [36]:
human_label = input("Review network data and classify attack:") 

Review network data and classify attack:DDoS


In [37]:
attacks['Label']=human_label

In [38]:
del attacks['Unnamed: 0']

In [39]:
attacks

,AttackId,Destination Port,Flow Duration,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,Bwd Packet Length Min,...,Init_Win_bytes_forward,Init_Win_bytes_backward,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Std,Idle Min,Label
614584,614584,53176,11492,4,24,6,6,6.0,0,0,...,180,-1,20,0.0,0.0,0,0,0.0,0,DDoS


In [40]:
real_time

,Unnamed: 0,AttackId,Destination Port,Flow Duration,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,...,Init_Win_bytes_forward,Init_Win_bytes_backward,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Std,Idle Min,Label
0,0,614567,53,215,2,76,38,38,38.0,242,...,-1,-1,20,0.0,0.0,0,0,0.0,0,BENIGN
1,1,614568,53,837,2,74,37,37,37.0,77,...,-1,-1,20,0.0,0.0,0,0,0.0,0,BENIGN
2,2,614569,443,14336951,1,0,0,0,0.0,0,...,8192,42780,32,6336908.0,0.0,6336908,6336908,0.0,8000043,BENIGN
3,3,614570,53,31040,2,82,41,41,41.0,113,...,-1,-1,32,0.0,0.0,0,0,0.0,0,BENIGN
4,4,614571,53,148,2,76,38,38,38.0,54,...,-1,-1,20,0.0,0.0,0,0,0.0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,428,614995,53,30477,1,79,79,79,79.0,145,...,-1,-1,32,0.0,0.0,0,0,0.0,0,BENIGN
429,429,614996,53,30572,1,46,46,46,46.0,74,...,-1,-1,32,0.0,0.0,0,0,0.0,0,BENIGN
430,430,614997,53,30473,1,60,60,60,60.0,76,...,-1,-1,20,0.0,0.0,0,0,0.0,0,BENIGN
431,431,614998,53,31267,1,48,48,48,48.0,113,...,-1,-1,20,0.0,0.0,0,0,0.0,0,BENIGN


###### **6.2.6 Add classified attack to training data and remove from analysis checklist**

In [41]:
all_data.append(attacks)
all_data.to_csv(r'clean_data.csv', index = True, header=True)

In [42]:
test=pd.read_csv('clean_data.csv')

In [43]:
test = test[test['AttackId']==614765]
del test
del test['Unnamed: 0']

In [44]:
test

,Unnamed: 0.1,AttackId,Destination Port,Flow Duration,Total Fwd Packets,Total Length of Fwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Bwd Packet Length Max,...,Init_Win_bytes_forward,Init_Win_bytes_backward,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Std,Idle Min,Label
614765,614765,614765,8080,1045016,3,0,0,0,0.0,6,...,8192,0,28,0.0,0.0,0,0,0.0,0,Bot


### Conclusion/ Workflow

> Trained a KNN-based classifier with difference distance metrics and arriving on a single model which performed best in terms of accuracy/ confidence rate and computation/ training time.

> Using the trained classifier, trying to classfify unlabeled attacks and alerting 'human' incase of low confidence/ prediction score. 

> Adding the 'manually' classified labeled data back to the training data for future classification and learning.

In a nutshell, trying to use Machine learning with safety(human assistance for critical actions) and transfering this learning back to the classifying model (Active/ Reinforcement learning).